In [ ]:
import os

In [ ]:
import requests

TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")
TOGETHER_API_URL = os.getenv("TOGETHER_API_URL")

def topic_breakdown(topic):
    messages = [
        {"role": "system", "content": "You are a helpful research assistant."},
        {"role": "user", "content": f"Please break down the following topic into five smaller research questions. Return as a list.\nTopic: {topic}"}
    ]

    headers = {
        "Authorization": f"Bearer {TOGETHER_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "meta-llama/Llama-3-8b-chat-hf",
        "messages": messages,
        # "max_tokens": 520,
        "temperature": 0.8
    }

    response = requests.post(TOGETHER_API_URL, headers=headers, json=payload)

    print(response.status_code)
    print(response.json())

    return response.json()["choices"][0]["message"]["content"]

topic = "How to enhance the personalization and diversity of news recommendation systems?"
print(topic_breakdown(topic))


200
{'id': 'nrKUPe7-4yUbBN-93522a587f5e4a39', 'object': 'chat.completion', 'created': 1745461687, 'model': 'meta-llama/Llama-3-8b-chat-hf', 'prompt': [], 'choices': [{'finish_reason': 'stop', 'seed': 6474932308487813000, 'logprobs': None, 'index': 0, 'message': {'role': 'assistant', 'content': 'Here are five smaller research questions to help break down the topic "How to enhance the personalization and diversity of news recommendation systems?":\n\n**1.** What are the current approaches to personalization in news recommendation systems, and how can they be improved to better incorporate users\' preferences, interests, and behaviors?\n\n**2.** How can natural language processing (NLP) and machine learning algorithms be leveraged to increase the diversity of news stories suggested to users, while still maintaining relevance and accuracy?\n\n**3.** What are the benefits and challenges of incorporating user feedback and ratings into news recommendation systems, and how can algorithms be de

In [ ]:
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")
TOGETHER_API_URL = os.getenv("TOGETHER_API_URL")
def expand_query(subquery):
    headers = {
        "Authorization": f"Bearer {TOGETHER_API_KEY}",
        "Content-Type": "application/json"
    }

    messages = [
        {"role": "system", "content": "You are a helpful research assistant."},
        {"role": "user", "content": f"""
You are given a research subquestion. Your job is to expand this query with related search keywords, synonyms, and relevant phrases.

- Provide 5-7 high-quality keywords or phrases.
- Return them in the form of a Python list.

Subquestion: {subquery}
"""}
    ]

    payload = {
        "model": "meta-llama/Llama-3-8b-chat-hf",
        "messages": messages,
        "max_tokens": 250,
        "temperature": 0.5
    }

    try:
        response = requests.post("https://api.together.xyz/v1/chat/completions", headers=headers, json=payload)
        response_json = response.json()


        if "choices" not in response_json:
            print("API Error:")
            print(response_json)
            return "Error"

        return response_json["choices"][0]["message"]["content"]

    except Exception as e:
        print("Exception occurred:", str(e))
        return "Error"


subquestions = [
    "What are the most effective machine learning algorithms for personalizing news recommendations, and how can they be adapted to incorporate user feedback and behavior?",
    "How can natural language processing (NLP) and topic modeling techniques be used to improve the diversity of news recommendations, and what are the most effective techniques for identifying and incorporating diverse perspectives?",
    "Can social media and user-generated content be used to enhance the personalization and diversity of news recommendations, and what are the most effective ways to integrate these sources into recommendation systems?",
    "What are the most effective ways to address concerns around echo chambers, filter bubbles, and the potential negative impacts of personalized news recommendations, and how can recommendation systems be designed to promote diverse and informed news consumption?",
    "How can human evaluative judgments and expert feedback be incorporated into the development and testing of news recommendation systems, and what are the most effective methods for assessing the quality and diversity of recommended news articles?"
]

for i, q in enumerate(subquestions, 1):
    print(f"\nSubquestion {i}: {q}")
    print("Expanded Keywords:")
    print(expand_query(q))



Subquestion 1: What are the most effective machine learning algorithms for personalizing news recommendations, and how can they be adapted to incorporate user feedback and behavior?
Expanded Keywords:
Here's an expanded list of keywords and phrases related to the subquestion:

```
research_keywords = [
    "Personalized news recommendation algorithms",
    "Machine learning for news personalization",
    "User feedback in news recommendation systems",
    "Behavioral analysis for news personalization",
    "Collaborative filtering for news recommendation",
    "Content-based filtering for news recommendation",
    "Hybrid approach for news recommendation",
    "News recommendation systems evaluation metrics"
]
```

These keywords and phrases can be used as search terms to find relevant research papers, articles, and studies on the topic. They cover various aspects of the subquestion, including:

1. Machine learning algorithms for personalizing news recommendations
2. Incorporating use

3. Search Tool: Create a wrapper around the You API or Brave Search API, Serper.dev. Please note that the free tier is 1000 queries/month. Consider creating a mock while developing, and switch to actually call the You API once the agent is more stable. Additionally, consider caching the search results.

In [ ]:
import os
if os.path.exists("search_cache.json"):
    os.remove("search_cache.json")
    print("Cleared old mock cache.")

In [ ]:
# import requests

# BRAVE_API_KEY = "BSA24rB-2SHK86awfy4ml4lSsGlCl25"
# BRAVE_SEARCH_URL = "https://api.search.brave.com/res/v1/web/search"

# def brave_search(query, count=5):
#     headers = {
#         "Accept": "application/json",
#         "Accept-Encoding": "gzip",
#         "X-Subscription-Token": BRAVE_API_KEY
#     }

#     params = {
#         "q": query,
#         "count": count
#     }

#     try:
#         response = requests.get(BRAVE_SEARCH_URL, headers=headers, params=params)
#         results = response.json().get("web", {}).get("results", [])
#         return [f"{item['title']} - {item['url']}" for item in results]

#     except Exception as e:
#         print("Search is failed:", str(e))
#         return []

import requests
import json
import os
import time
# Brave API config
BRAVE_API_KEY =os.getenv("BRAVE_API_KEY")
BRAVE_SEARCH_URL = "https://api.search.brave.com/res/v1/web/search"
CACHE_FILE = "search_cache.json"

# Load cache
if os.path.exists(CACHE_FILE):
    with open(CACHE_FILE, "r") as f:
        search_cache = json.load(f)
else:
    search_cache = {}

# Brave search with caching
def brave_search(query, count=3):
    # If in cache, return from cache
    if query in search_cache:
        return search_cache[query]

    # Else, call Brave API
    headers = {
        "Accept": "application/json",
        "Accept-Encoding": "gzip",
        "X-Subscription-Token": BRAVE_API_KEY
    }

    params = {
        "q": query,
        "count": count
    }

    try:
        response = requests.get(BRAVE_SEARCH_URL, headers=headers, params=params)
        response.raise_for_status()
        results = response.json().get("web", {}).get("results", [])
        formatted = [f"{item['title']} - {item['url']}" for item in results]

        # Save to cache
        search_cache[query] = formatted
        with open(CACHE_FILE, "w") as f:
            json.dump(search_cache, f, indent=2)

        return formatted

    except Exception as e:
        print("Search failed:", str(e))
        return []

# Run multiple queries
queries = [
    "personalized news recommendations",
    "machine learning algorithms for recommendation systems",
    "user feedback incorporation",
    "behavioral analysis for news personalization",
    "content-based filtering",
    "collaborative filtering techniques",
    "hybrid recommendation approaches"
]

for q in queries:
    print(f"\nSearching for: {q}")
    results = brave_search(q, count=3)
    for r in results:
        print("##", r)
        time.sleep(1)



Searching for: personalized news recommendations
## Personalized News Recommendation Based on Click ... - https://research.google.com/pubs/archive/35599.pdf
## Personalized News Recommendation Based on Click Behavior - https://research.google/pubs/pub35599/
## A Survey of Personalized News Recommendation | Data Science and Engineering - https://link.springer.com/article/10.1007/s41019-023-00228-5

Searching for: machine learning algorithms for recommendation systems
## 9 Machine Learning Algorithms For Recommendation Engines - https://insights.daffodilsw.com/blog/machine-learning-algorithms-for-recommendation-engines
## Machine Learning for Recommender systems — Part 1 (algorithms, evaluation and cold start) | by Pavel Kordík | Recombee blog | Medium - https://medium.com/recombee-blog/machine-learning-for-recommender-systems-part-1-algorithms-evaluation-and-cold-start-6f696683d0ed
## Recommender Systems - A Complete Guide to Machine Learning Models | Towards Data Science - https://tow

In [ ]:
import json

# Load the uploaded cache file
cache_path = "search_cache.json"
with open(cache_path, "r") as f:
    cache = json.load(f)

# Display the available topics (keys in the cache)
list(cache.keys())

['personalized news recommendations',
 'machine learning algorithms for recommendation systems',
 'user feedback incorporation',
 'behavioral analysis for news personalization',
 'content-based filtering',
 'collaborative filtering techniques',
 'hybrid recommendation approaches']

In [ ]:
# Tool 4: Summarizer
import requests
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")
TOGETHER_API_URL = os.getenv("TOGETHER_API_URL")
def summarize_brave_results(results, topic):
    content = "\n".join(results)

    prompt = f"""
You are a helpful research assistant.

Your task is to summarize the following search result titles and links into a 100-word paragraph that answers the topic below.

Topic: {topic}

Search Results:
{content}

Summarize clearly, concisely, and only based on the provided information.
"""

    headers = {
        "Authorization": f"Bearer {TOGETHER_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "meta-llama/Llama-3-8b-chat-hf",
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 250,
        "temperature": 0.5
    }

    response = requests.post(TOGETHER_API_URL, headers=headers, json=payload)
    return response.json()["choices"][0]["message"]["content"]

# Tool 5: Critique
def critique_summary(summary_text, topic):
    prompt = f"""
You are an expert reviewer. A student has written the following summary about the topic: "{topic}".

Please do the following:
1. Critique the clarity, completeness, and correctness of the summary.
2. Suggest specific ways the summary could be improved (e.g., add missing concepts, remove vague terms).
3. Recommend 1–2 additional topics or research angles related to this subject that would be useful to explore next.

Summary:
\"\"\"
{summary_text}
\"\"\"

Respond in clear bullet points, organized under the headings: "Evaluation", "Improvement Suggestions", and "Related Topics to Explore".
"""

    headers = {
        "Authorization": f"Bearer {TOGETHER_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "meta-llama/Llama-3-8b-chat-hf",
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 300,
        "temperature": 0.5
    }

    response = requests.post(TOGETHER_API_URL, headers=headers, json=payload)
    return response.json()["choices"][0]["message"]["content"]

# Get the search results for the topic
topic = "personalized news recommendations"
results = cache[topic]

# We won't run this here because it needs a real API key
# summary = summarize_brave_results(results, topic)
# critique = critique_summary(summary, topic)

summary = summarize_brave_results(results, topic)
critique = critique_summary(summary, topic)
print(summary,critique)

Here is a summary of the search results in 100 words:

Personalized news recommendations aim to provide users with relevant and engaging news content. Research has explored the use of click behavior to personalize news recommendations. A study by Google (Google Research, 2015) developed a personalized news recommendation system based on click data, achieving improved user engagement. A survey of personalized news recommendation systems (Data Science and Engineering, 2023) highlights the importance of considering user behavior, such as clicks and browsing history, to provide tailored news recommendations. These approaches can enhance user experience and increase news consumption. Here is the evaluation, improvement suggestions, and related topics to explore:

**Evaluation**

* Clarity: The summary is generally clear, but some sentences could be rephrased for better flow and coherence.
* Completeness: The summary provides a good overview of the topic, but could benefit from more specific

In [ ]:
# Full Research Agent Workflow Implementation
import json
import requests
import os
import time

# Configuration
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")
TOGETHER_API_URL = os.getenv("TOGETHER_API_URL")
BRAVE_API_KEY = os.getenv("BRAVE_API_KEY")
BRAVE_SEARCH_URL = "https://api.search.brave.com/res/v1/web/search"
CACHE_PATH = "search_cache.json"

# Load search result cache
if os.path.exists(CACHE_PATH):
    with open(CACHE_PATH, "r") as f:
        cache = json.load(f)
else:
    cache = {}

# Tool 1: Topic Breakdown Tool
# Use LLM to split a broad research topic into focused subtopics
def topic_breakdown(topic):
    prompt = f"""
You are a research assistant. Break down the following research topic into 3 to 5 focused subtopics.
Return only a valid JSON list of strings. No explanations, no formatting, no markdown.
Topic: {topic}
"""
    return call_llm(prompt)


# Tool 2: Query Expansion Tool
# Use LLM to generate relevant search phrases for a subtopic
def expand_query(subtopic):
    prompt = f"""
Generate 5 relevant search keywords or phrases to explore the topic:
\"{subtopic}\"
Return them as a list.
"""
    return call_llm(prompt)

# Tool 3: Brave Search Tool with Caching
# Perform Brave search and cache the results locally to avoid repeat queries
def brave_search(query, count=3):
    if query in cache:
        return cache[query]

    headers = {
        "Accept": "application/json",
        "X-Subscription-Token": BRAVE_API_KEY
    }
    params = {"q": query, "count": count}

    try:
        response = requests.get(BRAVE_SEARCH_URL, headers=headers, params=params)
        results = response.json().get("web", {}).get("results", [])
        formatted = [f"{item['title']} - {item['url']}" for item in results]
        cache[query] = formatted
        with open(CACHE_PATH, "w") as f:
            json.dump(cache, f, indent=2)
        return formatted
    except Exception as e:
        print("Search failed:", e)
        return []

# Tool 4: Summarizer Tool
# Summarize search results into a 100-word paragraph using an LLM
def summarize_brave_results(results, topic):
    content = "\n".join(results)
    prompt = f"""
Summarize the following web search results (titles and URLs) into a 100-word paragraph answering this topic:
{topic}
{content}
"""
    return call_llm(prompt)

# Tool 5: Critique Tool
# Evaluate the summary, suggest improvements, and recommend related topics
def critique_summary(summary, topic):
    prompt = f"""
Critique the following summary of \"{topic}\":
{summary}

Provide:
- Evaluation
- Suggestions for improvement
- Related topics to explore
"""
    return call_llm(prompt)

# Unified function to call LLM through Together API
# This is used by all tools that require natural language generation
def call_llm(prompt):
    headers = {
        "Authorization": f"Bearer {TOGETHER_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "meta-llama/Llama-3-8b-chat-hf",
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 300,
        "temperature": 0.5
    }

    try:
        response = requests.post("https://api.together.xyz/v1/chat/completions", headers=headers, json=payload)
        response.raise_for_status()
        data = response.json()
        if "choices" in data:
            return data["choices"][0]["message"]["content"].strip()
        else:
            print("Unexpected response format:", data)
            return "[Error] Invalid response format"
    except Exception as e:
        print("LLM request failed:", e)
        print("Response text:", response.text if 'response' in locals() else "No response")
        return "[Error] Failed to call LLM"


# Full agent workflow
# This function orchestrates the use of all tools to complete the assignment

def research_agent_workflow(topic):
    print("Main topic:", topic)

    raw_subtopics = topic_breakdown(topic)
    try:
        subtopics = json.loads(raw_subtopics) if isinstance(raw_subtopics, str) else raw_subtopics
        if not isinstance(subtopics, list) or not all(isinstance(s, str) for s in subtopics):
            raise ValueError
    except:
        print("Failed to parse subtopics. Using default.")
        subtopics = [topic]

    print("Subtopics:", subtopics)

    for sub in subtopics:
        print("\nSubtopic:", sub)

        raw_keywords = expand_query(sub)
        try:
            keywords = json.loads(raw_keywords) if isinstance(raw_keywords, str) else raw_keywords
            if not isinstance(keywords, list) or not all(isinstance(k, str) for k in keywords):
                raise ValueError
        except:
            print("Failed to parse keywords. Using subtopic only.")
            keywords = [sub]

        print("Keywords:", keywords)

        all_results = []
        for kw in keywords:
            print("Searching:", kw)
            results = brave_search(kw)
            all_results.extend(results)
            time.sleep(1.5)

        if not all_results:
            print("No results found.")
            continue

        summary = summarize_brave_results(all_results, sub)
        print("\nSummary:\n", summary)

        critique = critique_summary(summary, sub)
        print("\nCritique:\n", critique)


In [ ]:
research_agent_workflow("How to enhance the personalization and diversity of news recommendation systems?")

Main topic: How to enhance the personalization and diversity of news recommendation systems?
Subtopics: ['Enhancing User Profiling for Personalized News Recommendations', 'Exploring Novel Content Representation Techniques for Increased Diversity', 'Improving Context-Aware News Recommendation for Enhanced Relevance', ' Investigating Hybrid Approaches for Combining Multiple Recommendation Strategies', 'Optimizing News Recommendation Algorithms for Real-Time Processing and Scalability']

Subtopic: Enhancing User Profiling for Personalized News Recommendations
Failed to parse keywords. Using subtopic only.
Keywords: ['Enhancing User Profiling for Personalized News Recommendations']
Searching: Enhancing User Profiling for Personalized News Recommendations

Summary:
 Here is a 100-word summary of the web search results:

Enhancing user profiling is crucial for personalized news recommendations. A survey of personalized news recommendation systems highlights the importance of user profiling, 